# 对delicious进行数据处理与分析

使用SimpleTagBased算法对Delicious2K数据进行推荐
原始数据集：https://grouplens.org/datasets/hetrec-2011/
数据格式：userID     bookmarkID     tagID     timestamp

In [1]:
import pandas as pd
import numpy as np
import random
import math
import operator
from datetime import datetime

In [2]:
data = pd.read_table?

In [ ]:
data = pd.read_table

In [3]:
data = pd.read_table

### 数据处理与数据分析

In [4]:
data = pd.read_table('../datasets/user_taggedbookmarks-timestamps.dat')

In [5]:
data.info

<bound method DataFrame.info of         userID  bookmarkID  tagID      timestamp
0            8           1      1  1289255362000
1            8           2      1  1289255159000
2            8           7      1  1289238901000
3            8           7      6  1289238901000
4            8           7      7  1289238901000
...        ...         ...    ...            ...
437588  108035       30993    193  1277495315000
437589  108035       30993    673  1277495315000
437590  108035       30994    130  1277223715000
437591  108035       30994    267  1277223715000
437592  108035       30994   4943  1277223715000

[437593 rows x 4 columns]>

In [6]:
data.columns

Index(['userID', 'bookmarkID', 'tagID', 'timestamp'], dtype='object')

In [7]:
data.head()

,userID,bookmarkID,tagID,timestamp
0,8,1,1,1289255362000
1,8,2,1,1289255159000
2,8,7,1,1289238901000
3,8,7,6,1289238901000
4,8,7,7,1289238901000


In [8]:
def timestamp_to_strtime(timestamp):
    '''
    将 13 位整数的毫秒时间戳转化成本地普通时间 (字符串格式)
    :param timestamp: 13 位整数的毫秒时间戳 (1456402864242)
    :return: 返回字符串格式 {str}'2016-02-25 20:21:04.242000'
    ''' 
    
    local_str_time = datetime.fromtimestamp(int(timestamp) / 1000.0).strftime('%Y-%m-%d %H:%M:%S')
    
    return local_str_time

In [9]:
#测试
timestamp_to_strtime(int(data.loc[1,'timestamp']))

'2010-11-09 06:25:59'

In [10]:
#对DataFrame进行时间格式处理(map函数)
data['date_format']=data['timestamp'].map(timestamp_to_strtime)

In [11]:
data.head()

,userID,bookmarkID,tagID,timestamp,date_format
0,8,1,1,1289255362000,2010-11-09 06:29:22
1,8,2,1,1289255159000,2010-11-09 06:25:59
2,8,7,1,1289238901000,2010-11-09 01:55:01
3,8,7,6,1289238901000,2010-11-09 01:55:01
4,8,7,7,1289238901000,2010-11-09 01:55:01


In [12]:
#把之前的string类型的时间数据转换datetime时间对象。
data['date_format'] = pd.to_datetime(data['date_format'])

In [13]:
data['date_format'].dtype

dtype('<M8[ns]')

In [14]:
dict.setdefault?

In [15]:
#用户8给物品7，打过哪些标签
f = lambda x: (x['userID'] == 8) and (x['bookmarkID'] == 7)

In [16]:
data.loc[0]['userID']

8

In [17]:
#多行值判断搜索
data[data.apply(f, axis= 1)]

,userID,bookmarkID,tagID,timestamp,date_format
2,8,7,1,1289238901000,2010-11-09 01:55:01
3,8,7,6,1289238901000,2010-11-09 01:55:01
4,8,7,7,1289238901000,2010-11-09 01:55:01


In [25]:
#按照行进行迭代
for _index, _content in data.loc[1:3,:].iterrows():
    print (_content['bookmarkID'])

2
7
7


### 分步算法实现

1、将数据载入形成嵌套字典格式。

In [158]:
#数据加载,把数据转换成records字典。
def loadData(filename):
    #保存用户user对item的tag  格式{'uid':{'iid':[tag]}}
    records = {}
    data = pd.read_table(filename)
    for rowID, rowData in data.iterrows():
        userID = rowData['userID']
        itemID = rowData['bookmarkID']
        tagID = rowData['tagID']
        
        #若不存在则默认处理
        records.setdefault(userID,{})
        records[userID].setdefault(itemID,[])
        records[userID][itemID].append(tagID)
        
    return records
        
        

In [159]:
filename = '../datasets/user_taggedbookmarks-timestamps.dat'
records = loadData(filename)

In [160]:
records.keys()

dict_keys([8, 32, 57, 147, 233, 242, 259, 302, 319, 336, 372, 427, 485, 521, 538, 620, 788, 819, 913, 934, 970, 1016, 1033, 1049, 1094, 1103, 1113, 1149, 1263, 1296, 1320, 1328, 1352, 1423, 1455, 1476, 1505, 1557, 1701, 1751, 1774, 1822, 1856, 1887, 1933, 1952, 1987, 2022, 2032, 2055, 2128, 2151, 2232, 2300, 2315, 2330, 2367, 2393, 2437, 2462, 2487, 2498, 2549, 2570, 2586, 2611, 2636, 2701, 2716, 2748, 2831, 2879, 2911, 2934, 2952, 2961, 2989, 3032, 3083, 3122, 3203, 3220, 3233, 3257, 3279, 3337, 3396, 3421, 3454, 3513, 3539, 3568, 3646, 3728, 3743, 3789, 3817, 3836, 3914, 3943, 3957, 4031, 4089, 4109, 4135, 4154, 4188, 4237, 4324, 4363, 4488, 4549, 4681, 4717, 4742, 4762, 4795, 4828, 4911, 4977, 5024, 5046, 5075, 5108, 5186, 5251, 5266, 5323, 5424, 5478, 5584, 5606, 5648, 5802, 5817, 5863, 5937, 5947, 5959, 6020, 6047, 6067, 6092, 6111, 6120, 6141, 6157, 6186, 6195, 6263, 6403, 6452, 6510, 6548, 6585, 6652, 6663, 6677, 6704, 6796, 6862, 6888, 6897, 6919, 6934, 6995, 7050, 7066, 7081, 

2、对数据进行分割，分成测试集30%与训练集70%

In [161]:
#将数据集及训练集进行拆分，分割成训练集与测试集
#数据集分割要按照用户对item的喜欢维度进行。即用户喜欢商品的一部分作为测试集，
#一部分作为训练集, 用测试集预测算法的准确性。

def randomlySplitData(records = records, radio = 0.3):
    random.seed(100)
    trainData = {}
    testData = {}
    for u in records.keys():
        for i in records[u].keys():
            if random.random() < radio:
                testData.setdefault(u, {})
                testData[u].setdefault(i, [])
                for v in records[u][i]:
                    testData[u][i].append(v)
            else:
                trainData.setdefault(u, {})
                trainData[u].setdefault(i, [])
                for v in records[u][i]:
                    trainData[u][i].append(v)
    return trainData, testData        

In [162]:
train, test = randomlySplitData()

In [163]:
len(test[8]),len(train[8])

(16, 54)

3、训练集进行初始化成三个字典的数据：user_tags, tag_items, user_items

In [164]:
def initStat(data : dict):
    
    user_tags = {}
    tag_items = {}
    user_items = {}
    for u, items in data.items():
        for i, tags in items.items():
            for t in tags:
                _addValueToMat(user_tags, u, t, 1)
                _addValueToMat(tag_items, t, i, 1)
                _addValueToMat(user_items, u, i, 1)
    return user_tags, tag_items, user_items

#辅助函数，设置嵌套字典,计算值，user_tags,tag_items, user_items
def _addValueToMat(mat, index, item, defaultValue = 1):
    #{key: {item : value}}
    if index not in mat:
        mat.setdefault(index, {})
        mat[index].setdefault(item, defaultValue)
    else:
        if item not in mat[index]:
            mat[index][item] = defaultValue
        else:
            mat[index][item] += defaultValue
    return mat
    
    

In [165]:
user_tags, tag_items, user_items = initStat(train)

In [166]:
user_tags.keys()

dict_keys([8, 32, 57, 147, 233, 242, 259, 302, 319, 336, 372, 427, 485, 521, 538, 620, 788, 819, 913, 934, 970, 1016, 1033, 1049, 1094, 1103, 1113, 1149, 1263, 1296, 1320, 1328, 1352, 1423, 1455, 1476, 1505, 1557, 1701, 1751, 1774, 1822, 1856, 1887, 1933, 1952, 1987, 2022, 2032, 2055, 2128, 2151, 2232, 2300, 2315, 2330, 2367, 2393, 2437, 2462, 2487, 2498, 2549, 2570, 2586, 2611, 2636, 2701, 2716, 2748, 2831, 2879, 2911, 2934, 2952, 2961, 2989, 3032, 3083, 3122, 3203, 3220, 3233, 3257, 3279, 3337, 3396, 3421, 3454, 3513, 3539, 3568, 3646, 3728, 3743, 3789, 3817, 3836, 3914, 3943, 3957, 4031, 4089, 4109, 4135, 4154, 4188, 4237, 4324, 4363, 4488, 4549, 4681, 4717, 4742, 4762, 4795, 4828, 4911, 4977, 5024, 5046, 5075, 5108, 5186, 5251, 5266, 5323, 5424, 5478, 5584, 5606, 5648, 5802, 5817, 5863, 5937, 5947, 5959, 6020, 6047, 6067, 6092, 6111, 6120, 6141, 6157, 6186, 6195, 6263, 6403, 6452, 6510, 6548, 6585, 6652, 6663, 6677, 6704, 6796, 6862, 6888, 6897, 6919, 6934, 6995, 7050, 7066, 7081, 

In [167]:
tag_items

{1: {2: 1,
  7: 1,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  14: 1,
  16: 1,
  18: 1,
  19: 1,
  20: 1,
  22: 1,
  24: 1,
  30: 1,
  32: 1,
  33: 1,
  35: 1,
  36: 1,
  37: 1,
  38: 1,
  61: 1,
  1: 1,
  15: 1,
  588: 1,
  661: 1,
  665: 1,
  855: 1,
  909: 1,
  918: 1,
  935: 1,
  936: 1,
  953: 1,
  1296: 1,
  1330: 1,
  1337: 1,
  1346: 1,
  1417: 1,
  1679: 1,
  1681: 1,
  1815: 1,
  1828: 1,
  1848: 1,
  1867: 1,
  1874: 1,
  1875: 1,
  1881: 1,
  1886: 1,
  1888: 1,
  1890: 1,
  2083: 1,
  2085: 1,
  2631: 1,
  2757: 1,
  2786: 1,
  3132: 1,
  3133: 1,
  3136: 1,
  3162: 1,
  3188: 1,
  3203: 1,
  3220: 1,
  3227: 1,
  4083: 1,
  4122: 1,
  4189: 1,
  4243: 1,
  4571: 1,
  4638: 1,
  4639: 2,
  4645: 1,
  4707: 1,
  4737: 1,
  4535: 1,
  5085: 1,
  6411: 1,
  6427: 1,
  6471: 1,
  6508: 1,
  6651: 1,
  6659: 1,
  6668: 1,
  6679: 1,
  6688: 1,
  6713: 1,
  6886: 1,
  6888: 1,
  6891: 1,
  6924: 1,
  7166: 1,
  1398: 1,
  7614: 1,
  7618: 1,
  7624: 1,
  7628: 1,
  7635: 1,
  7637: 1,
 

In [168]:
#检查
train[8]

{2: [1],
 7: [1, 6, 7],
 8: [1, 8, 9],
 9: [1, 10],
 10: [1, 11],
 11: [1, 12, 13],
 14: [1, 15, 16, 17, 18],
 16: [1, 5],
 18: [1, 21],
 19: [1, 22],
 20: [1, 23],
 22: [1, 10, 25],
 24: [1, 27],
 30: [1, 33],
 32: [1, 35],
 33: [1, 36],
 35: [1, 38],
 36: [1, 39],
 37: [1, 40],
 38: [1, 41],
 42: [43],
 45: [46],
 46: [47],
 48: [49, 50, 51, 52],
 53: [51, 58, 59, 60, 61],
 58: [2, 66],
 60: [2, 67],
 61: [1, 68],
 62: [69],
 64: [71],
 65: [71],
 68: [45, 76, 77],
 71: [44],
 72: [45, 79],
 73: [24, 56, 80],
 75: [45, 76, 84],
 76: [45, 76],
 78: [45, 76, 86, 87],
 79: [45, 88, 89, 90, 91, 92, 93],
 80: [45, 88, 94, 95],
 81: [96, 97],
 82: [68, 98, 99, 100],
 83: [2, 101, 102],
 84: [2, 103],
 85: [104, 105],
 88: [2, 25, 66, 82, 106],
 89: [82, 109],
 90: [24],
 91: [24],
 92: [24],
 93: [24, 25, 66],
 94: [2],
 98: [112],
 99: [113]}

4、对用户推荐top-N,并排序

In [173]:
def recommend(user, N, user_tags, tag_items, user_items):
    recomend_items = {}
    used_items = user_items[user]
    #print(used_items)
    for tag, v_ui in user_tags[user].items():
        for item, v_ti in tag_items[tag].items():
            if item in used_items:   #已经买过的物品不计算兴趣值
                continue
            if item not in recomend_items:
                recomend_items[item] = v_ui * v_ti
            else:
                recomend_items[item] += v_ui * v_ti
    return sorted(recomend_items.items(), key = operator.itemgetter(1), reverse=True)[0:N]
        
    

In [174]:
item_r = recommend(8, 4, user_tags, tag_items, user_items)

In [175]:
item_r

[(1416, 55), (1526, 46), (4639, 42), (4535, 42)]

5、使用测试集，计算准确率和召回率

In [177]:
def precisionAndRecall(test, N, train):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user, items in test.items():
        if user not in train:
            continue
        #获取top-n 推荐列表
        rank = recommend(user, N,user_tags, tag_items, user_items)
        for item, rui in rank:
            if item in items:
                hit += 1
        h_recall += len(items)
        h_precision += N
    return (hit/(h_precision * 1.0)),(hit/(h_recall * 1.0))
            
        

In [178]:
precision, recall = precisionAndRecall(test, 10, train)

In [179]:
precision

0.00920687742651137

In [180]:
recall

0.005299112558258315